# Solving differential equations system 

To solve differential equation system it is again nessecary to perform the following workflow:

1. Define Models
2. Create residual function
3. Find the optimal constants using lmfit


## Step 1 defining the models

The basic procedure is the same, but there are differences:

- The initial conditions (**y0_array**) are defined as array and NOT as scalar
- There are two differential equations defined and **returned** as array

```
y0_array = [100,0]

def dglsys(init_cond_array, t,A0, kA, kB):
    A = init_cond_array[0]
    P = init_cond_array[1]
    dAdt = -kA*A
    dPdt = kB*(A0-(A0-A))
    dzdt = [dAdt, dPdt]
    return dzdt

```

## Step 2 defining the residual function

The residual function calculates the difference between the measured and the modelled values. Of course we need to create the `lmfit.Parameters()` object with the to be determined constants In the case of a **dgl-system** we will receive an array of integrated values and therefore need to also compare arrays with one another:

```
params = lmfit.Parameters()
params.add('kA', 2, min=0.00000001, max=3)
params.add('kP', 2, min=0.0000000001, max=3)

def residual(params, time, data_sets):
    kA = params['kA'].value
    kP = params['kP'].value
    ymodel=odeint(dglsys, y0_array, time, args=(100, kA, kP))
    substr = data_sets[0]-ymodel[:,0] # array 1
    product = data_sets[1]-ymodel[:,1] # array 2
    result = [substr, product] # result is returned as array
    return np.array(result)

```

3. Find the optimal constants using lmfit-minimize

And finally we can fit the constants by the minimize function of lmfit:

- In this case the data array will consist of two columns since we have to fit two different parameters

`solution = lmfit.minimize(residual, params, args=(np.array(time), data))`





In [2]:
from scipy.integrate import odeint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import lmfit

df = pd.read_excel(r"reaktionen 1 2 ordnung.xlsx")

print(df.head())


time = df["Zeit"]
substrate = df["Substratkonzentration k1"]
product = df["Produktkonzentration k1"]


fix, ax = plt.subplots()

ax.plot(time, substrate, label="Substrate concentration")
ax.plot(time, product, label="product concentration")
#ax.legend()

#plt.show()

ModuleNotFoundError: No module named 'lmfit'

In [1]:
# Definition of DGL system
# A = Initial conditions
# t = time or x-values
# kA = Constant A
# kB = Constant B
y0 = 100



data_list = [substrate, product]

data = np.array(data_list)



y0_array = [100,0, 0]

def dglsys(init_cond_array, t,A0, kA, kB, kC):
    A = init_cond_array[0]
    P = init_cond_array[1]
    P1 = init_cond_array[2]
    dAdt = -kA*A
    dPdt = kB*(A0-(A0-A))-kC*(kB*(A0-(A0-A)))
    dP1dt =kC*(kB*(A0-(A0-A)))
    dzdt = [dAdt, dPdt, dP1dt]
    return dzdt

sol = odeint(dglsys, y0_array, time, args=(100,0.2, 0.2, 0,1))
#print(sol)
print(np.array([time]).shape)
#print(data[0])

'''
params = lmfit.Parameters()
params.add('kA', 2, min=0.00000001, max=3)
params.add('kP', 2, min=0.0000000001, max=3)

def residual(params, time, data_sets):
    kA = params['kA'].value
    kP = params['kP'].value
    ymodel=odeint(dglsys, y0_array, time, args=(100, kA, kP))
    substr = data_sets[0]-ymodel[:,0] # array 1
    product = data_sets[1]-ymodel[:,1] # array 2
    result = [substr, product] # result is returned as array
    return np.array(result)

solution = lmfit.minimize(residual, params, args=(np.array(time), data))
print(lmfit.report_fit(solution))
'''

fig, ax = plt.subplots()
ax.plot(time, substrate, label="Substrate concentration", color="red")
ax.plot(time, sol[:,1], label="modelled")
ax.plot(time, sol[:,0], label="modelled substrate")
ax.legend()

plt.show()

NameError: name 'substrate' is not defined